In [2]:
%%time

# Import Packages
import pandas as pd
import numpy as np

import gapandas4 as gp # See Footnote 1 bottom of page
import os
import datetime
from datetime import date
# Written functions

import sys
# adding Notebooksfolder to the system path
sys.path.insert(0, '/Users/emil/miniforge3/envs/googleapi/Notebooks')

import importlib
import dates_funcs
importlib.reload(dates_funcs)
from dates_funcs import appendDFToCSV

# ---------------------------------------------------------------------------------------------------------------------

# Now to print to log when program running 
nowDT = datetime.datetime.now()
now = datetime.datetime.strftime(nowDT,'%Y-%m-%d_%H:%M:%S')
print("Script started: "+now)
print("\n")

# Check of Dates
f = open("latestDate.txt")
maxSavedDate = f.read()
f.close()
#print("Previously fetched data up to and including: "+ maxSavedDate)
maxSavedDateDT = datetime.datetime.strptime(maxSavedDate,'%Y-%m-%d').date()
print("Last date saved: "+maxSavedDate)
print("\n")

# Todays date
todayDT = date.today()
today = datetime.datetime.strftime(todayDT,'%Y-%m-%d')

# start_date as the next day as maxSavedDate
start_dateDT = maxSavedDateDT + datetime.timedelta(days=1)
start_date = datetime.datetime.strftime(start_dateDT,'%Y-%m-%d')
print("Fetching new data, starting: "+start_date)

# # end_date as the same day as start_date, to get data for one days
#end_dateDT = start_dateDT + datetime.timedelta(days=1)
end_dateDT = start_dateDT
end_date = datetime.datetime.strftime(end_dateDT,'%Y-%m-%d')
print("up to and including: "+end_date)
print("\n")

# ---------------------------------------------------------------------------------------------------------------------

#MASTER IF

if start_dateDT > maxSavedDateDT and todayDT > end_dateDT and todayDT > maxSavedDateDT:
    
    # START BACKUP or previous .csv files to an archive
    # DISABLED because taking up lots of space, will enable manually once in a while instead
    '''
    import pathlib
    import zipfile
    from zipfile import ZipFile, ZIP_LZMA
    
    directory = pathlib.Path("output/")

    try:
        with ZipFile("backup_date:_"+maxSavedDate+"_written:_"+now+".zip", mode="w",compression=ZIP_LZMA, allowZip64=True) as archive:
            for file_path in directory.rglob("*"):
                archive.write(
                    file_path,
                    arcname=file_path.relative_to(directory)
                )
            print("Previous .csv-files backed up to: backup_until:_"+maxSavedDate+"_written:_"+now+".zip")
    except BadZipFile as error:
        print(error)
    '''
    # END BACKUP    

    # ---------------------------------------------------------------------------------------------------------------------
    
    # CREDENTIALS
    property_id = "298727788"
    credentials_json_path="/Users/emil/miniforge3/envs/googleapi/Notebooks/Analytics/client_secrets.json"
    service_account = credentials_json_path
    
    # ---------------------------------------------------------------------------------------------------------------------
    
    # LISTS DIMENSIONS AND METRICS
    # Dimensions
    dimensions = ["language","languageCode","browser","deviceCategory","mobileDeviceBranding",
                  "mobileDeviceMarketingName","mobileDeviceModel","operatingSystemWithVersion",
                  "platform","screenResolution","firstUserDefaultChannelGroup","firstUserSource",
                  "sessionDefaultChannelGroup","sessionSource","eventName","pagePath","linkUrl",
                  "landingPage","pageReferrer"]
    
    # Does not work with cityId, only countryId
    # does not work with dateHour only date.
    dimensionsSpecial = ["userGender","brandingInterest"]
    
    # Does not work with cityId, only countryId,
    # does not work with dateHour only date:
    # also cant request cityId and date in same request.
    dimensionsSpecialSpecial = ["userAgeBracket"]
    
    dateHourList = ["dateHour"]
    dateList = ["date"]
    cityIdList = ["cityId"]
    countryIdList = ["countryId"]
    
    # Metrics with empty first place and repeat last to count 1-18
    metricList = ["","totalUsers","newUsers","activeUsers","userEngagementDuration","scrolledUsers",
                  "averageSessionDuration","bounceRate","engagedSessions","engagementRate","sessions",
                  "sessionsPerUser","eventCount","eventCountPerUser","eventsPerSession","screenPageViews",
                  "screenPageViewsPerSession","screenPageViewsPerUser","totalUsers"]
        
    loyaltyList = ["wauPerMau","dauPerMau","dauPerWau"]
        
    activityList = ["active28DayUsers","active7DayUsers","activeUsers"]
    
    # ---------------------------------------------------------------------------------------------------------------------
    
    # REQUESTS
    
    ### dimensions
    
    #### metricList
    
    # Empty list to fill with dataframes
    dfs = [[],[],[],]
    
    # Request for all dimensions in list
    for x in range(len(dimensions)):
        for y, z in zip([1,7,13],[0,1,2]): 
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateHourList[0]),
                    gp.Dimension(name=cityIdList[0]),
                    gp.Dimension(name=dimensions[x])
                ],
                metrics=[
                    gp.Metric(name=metricList[y]),
                    gp.Metric(name=metricList[y+1]),
                    gp.Metric(name=metricList[y+2]),
                    gp.Metric(name=metricList[y+3]),
                    gp.Metric(name=metricList[y+4]),
                    gp.Metric(name=metricList[y+5]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs[z].append(gp.query(service_account, report_request, report_type="report"))
    
            
    #### loyaltyList
    
    # Empty list to fill with dataframes
    dfs_l = []
    
    # Request for all dimensions in list
    for x in range(len(dimensions)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=cityIdList[0]),
                    gp.Dimension(name=dimensions[x])
                ],
                metrics=[
                    gp.Metric(name=loyaltyList[0]),
                    gp.Metric(name=loyaltyList[1]),
                    gp.Metric(name=loyaltyList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_l.append(gp.query(service_account, report_request, report_type="report"))
    
        
    #### activityList
    
    # Empty list to fill with dataframes
    dfs_a = []
    
    # Request for all dimensions in list
    for x in range(len(dimensions)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=cityIdList[0]),
                    gp.Dimension(name=dimensions[x])
                ],
                metrics=[
                    gp.Metric(name=activityList[0]),
                    gp.Metric(name=activityList[1]),
                    gp.Metric(name=activityList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_a.append(gp.query(service_account, report_request, report_type="report"))
    
    
        
    ### dimensionsSpecial
    
    # Empty list to fill with dataframes
    dfs_s = [[],[],[],]
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecial)):
        for y, z in zip([1,7,13],[0,1,2]): 
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=metricList[y]),
                    gp.Metric(name=metricList[y+1]),
                    gp.Metric(name=metricList[y+2]),
                    gp.Metric(name=metricList[y+3]),
                    gp.Metric(name=metricList[y+4]),
                    gp.Metric(name=metricList[y+5]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_s[z].append(gp.query(service_account, report_request, report_type="report"))
    
            
    #### loyaltyList
    
    # Empty list to fill with dataframes
    dfs_s_l = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=loyaltyList[0]),
                    gp.Metric(name=loyaltyList[1]),
                    gp.Metric(name=loyaltyList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_s_l.append(gp.query(service_account, report_request, report_type="report"))
    
        
    #### activityList
    
    # Empty list to fill with dataframes
    dfs_s_a = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=activityList[0]),
                    gp.Metric(name=activityList[1]),
                    gp.Metric(name=activityList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_s_a.append(gp.query(service_account, report_request, report_type="report"))
    
    
        
    ### dimensionsSpecialSpecial date
    
    #### metricList
    
    # Empty list to fill with dataframes
    dfs_ssd = [[],[],[],]
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
        for y, z in zip([1,7,13],[0,1,2]): 
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=metricList[y]),
                    gp.Metric(name=metricList[y+1]),
                    gp.Metric(name=metricList[y+2]),
                    gp.Metric(name=metricList[y+3]),
                    gp.Metric(name=metricList[y+4]),
                    gp.Metric(name=metricList[y+5]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssd[z].append(gp.query(service_account, report_request, report_type="report"))
    
            
    #### loyaltyList
    
    # Empty list to fill with dataframes
    dfs_ssd_l = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=loyaltyList[0]),
                    gp.Metric(name=loyaltyList[1]),
                    gp.Metric(name=loyaltyList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssd_l.append(gp.query(service_account, report_request, report_type="report"))
    
        
    #### activityList
    
    # Empty list to fill with dataframes
    dfs_ssd_a = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=dateList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=activityList[0]),
                    gp.Metric(name=activityList[1]),
                    gp.Metric(name=activityList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssd_a.append(gp.query(service_account, report_request, report_type="report"))
    
    
        
    ### dimensionsSpecialSpecial countryId
    
    #### metricList
    
    # Empty list to fill with dataframes
    dfs_ssc = [[],[],[],]
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
        for y, z in zip([1,7,13],[0,1,2]): 
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=metricList[y]),
                    gp.Metric(name=metricList[y+1]),
                    gp.Metric(name=metricList[y+2]),
                    gp.Metric(name=metricList[y+3]),
                    gp.Metric(name=metricList[y+4]),
                    gp.Metric(name=metricList[y+5]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssc[z].append(gp.query(service_account, report_request, report_type="report"))
    
            
    #### loyaltyList
    
    # Empty list to fill with dataframes
    dfs_ssc_l = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=loyaltyList[0]),
                    gp.Metric(name=loyaltyList[1]),
                    gp.Metric(name=loyaltyList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssc_l.append(gp.query(service_account, report_request, report_type="report"))
    
        
    #### activityList
    
    # Empty list to fill with dataframes
    dfs_ssc_a = []
    
    # Request for all dimensions in list
    for x in range(len(dimensionsSpecialSpecial)):
    
            report_request = gp.RunReportRequest(
                property=f"properties/{property_id}",
                dimensions=[
                    gp.Dimension(name=countryIdList[0]),
                    gp.Dimension(name=dimensionsSpecialSpecial[x])
                ],
                metrics=[
                    gp.Metric(name=activityList[0]),
                    gp.Metric(name=activityList[1]),
                    gp.Metric(name=activityList[2]),
                ],
                date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
            )
            
            # Perform query and append to list
            dfs_ssc_a.append(gp.query(service_account, report_request, report_type="report"))
    
    # ---------------------------------------------------------------------------------------------------------------------
        
    ## Formatting
    
    #### metricList
    
    # Drop the extra Totalusers column, since it exists twice in the tables to merge
    # since metricList contained it twice.
    
    for x in range(len(dimensions)):
        for z in [2]:
            dfs[z][x].drop(columns=['totalUsers'],inplace=True)
    
    for x in range(len(dimensionsSpecial)):
        for z in [2]:
            dfs_s[z][x].drop(columns=['totalUsers'],inplace=True)
    
    for x in range(len(dimensionsSpecialSpecial)):
        for z in [2]:
            dfs_ssd[z][x].drop(columns=['totalUsers'],inplace=True)
    
    for x in range(len(dimensionsSpecialSpecial)):
        for z in [2]:
            dfs_ssc[z][x].drop(columns=['totalUsers'],inplace=True)
            
    # ---------------------------------------------------------------------------------------------------------------------
        
    ## Merge
    
    dfs_merged = list(range(len(dimensions)))
    
    for x in range(len(dimensions)):
        mergeOn = ['dateHour','cityId'] 
        mergeDim = [dimensions[x]]
        mergeOn =  mergeOn + mergeDim
        dfs_merged[x] = dfs[0][x].merge(dfs[1][x], how='outer', on=mergeOn)
    
    for n in range(len(dimensions)):
        mergeOn = ['dateHour','cityId'] 
        mergeDim = [dimensions[n]]
        mergeOn =  mergeOn + mergeDim
        dfs_merged[n] = dfs_merged[n].merge(dfs[2][n], how='outer', on=mergeOn)
    
    
    dfs_s_merged = list(range(len(dimensionsSpecial)))
    
    for x in range(len(dimensionsSpecial)):
        mergeOn = ['date','countryId'] 
        mergeDim = [dimensionsSpecial[x]]
        mergeOn =  mergeOn + mergeDim
        dfs_s_merged[x] = dfs_s[0][x].merge(dfs_s[1][x], how='outer', on=mergeOn)
    
    for n in range(len(dimensionsSpecial)):
        mergeOn = ['date','countryId'] 
        mergeDim = [dimensionsSpecial[n]]
        mergeOn =  mergeOn + mergeDim
        dfs_s_merged[n] = dfs_s_merged[n].merge(dfs_s[2][n], how='outer', on=mergeOn)
    
    
    dfs_ssd_merged = list(range(len(dimensionsSpecialSpecial)))
    
    for x in range(len(dimensionsSpecialSpecial)):
        mergeOn = ['date'] 
        mergeDim = [dimensionsSpecialSpecial[x]]
        mergeOn =  mergeOn + mergeDim
        dfs_ssd_merged[x] = dfs_ssd[0][x].merge(dfs_ssd[1][x], how='outer', on=mergeOn)
    
    for n in range(len(dimensionsSpecialSpecial)):
        mergeOn = ['date'] 
        mergeDim = [dimensionsSpecialSpecial[n]]
        mergeOn =  mergeOn + mergeDim
        dfs_ssd_merged[n] = dfs_ssd_merged[n].merge(dfs_ssd[2][n], how='outer', on=mergeOn)
    
    
    dfs_ssc_merged = list(range(len(dimensionsSpecialSpecial)))
    
    for x in range(len(dimensionsSpecialSpecial)):
        mergeOn = ['countryId'] 
        mergeDim = [dimensionsSpecialSpecial[x]]
        mergeOn =  mergeOn + mergeDim
        dfs_ssc_merged[x] = dfs_ssc[0][x].merge(dfs_ssc[1][x], how='outer', on=mergeOn)
    
    for n in range(len(dimensionsSpecialSpecial)):
        mergeOn = ['countryId'] 
        mergeDim = [dimensionsSpecialSpecial[n]]
        mergeOn =  mergeOn + mergeDim
        dfs_ssc_merged[n] = dfs_ssc_merged[n].merge(dfs_ssc[2][n], how='outer', on=mergeOn)

    # ---------------------------------------------------------------------------------------------------------------------

    # Sort out dates and sort columns.
        
    #### metricList
    
    for x in range(len(dimensions)):
        for z in [0,1,2]: 
            dfs[z][x] = dates_funcs.sortOut_dateHour_short(dfs[z][x])
    
    for x in range(len(dimensionsSpecial)):
        for z in [0,1,2]: 
            dfs_s[z][x] = dates_funcs.sortOut_date_short(dfs_s[z][x])
            
    for x in range(len(dimensionsSpecialSpecial)):
        for z in [0,1,2]: 
            dfs_ssd[z][x] = dates_funcs.sortOut_date_short(dfs_ssd[z][x])
    
    #### loyaltyList
    
    for x in range(len(dimensions)):
        dfs_l[x] = dates_funcs.sortOut_date_short(dfs_l[x])
    
    for x in range(len(dimensionsSpecial)):
        dfs_s_l[x] = dates_funcs.sortOut_date_short(dfs_s_l[x])
            
    for x in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_l[x] = dates_funcs.sortOut_date_short(dfs_ssd_l[x])
    
    #### activityList
    
    for x in range(len(dimensions)):
        dfs_a[x] = dates_funcs.sortOut_date_short(dfs_a[x])
    
    for x in range(len(dimensionsSpecial)):
        dfs_s_a[x] = dates_funcs.sortOut_date_short(dfs_s_a[x])
            
    for x in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_a[x] = dates_funcs.sortOut_date_short(dfs_ssd_a[x])
    
    for x in range(len(dimensionsSpecialSpecial)):
        for z in [0,1,2]: 
            sortOrder = True
            dfs_ssc[z][x] = dfs_ssc[z][x].sort_values(list(dfs_ssc[z][x].columns.values), ascending=sortOrder)
            
    for x in range(len(dimensionsSpecialSpecial)):
        sortOrder = True
        dfs_ssc_l[x] = dfs_ssc_l[x].sort_values(list(dfs_ssc_l[x].columns.values), ascending=sortOrder)

            for x in range(len(dimensionsSpecialSpecial)):
        sortOrder = True
        dfs_ssc_a[x] = dfs_ssc_a[x].sort_values(list(dfs_ssc_a[x].columns.values), ascending=sortOrder)

        
    # ---------------------------------------------------------------------------------------------------------------------
        
    ## Sort out NA
    
    #### metricList
    
    dfs_merged_fillNA = list(range(len(dimensions)))
    
    for n in range(len(dimensions)):
        dfs_merged_fillNA[n] = dfs_merged[n].replace('', np.nan)
        dfs_merged_fillNA[n] = dfs_merged_fillNA[n].replace('(not set)', np.nan)
        print(dfs_merged_fillNA[n].shape)
    
    
    dfs_s_merged_fillNA = list(range(len(dimensionsSpecial)))
    
    for n in range(len(dimensionsSpecial)):
        dfs_s_merged_fillNA[n] = dfs_s_merged[n].replace('', np.nan)
        dfs_s_merged_fillNA[n] = dfs_s_merged_fillNA[n].replace('(not set)', np.nan)
        print(dfs_s_merged_fillNA[n].shape)
    
    
    dfs_ssd_merged_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_merged_fillNA[n] = dfs_ssd_merged[n].replace('', np.nan)
        dfs_ssd_merged_fillNA[n] = dfs_ssd_merged_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssd_merged_fillNA[n].shape)
    
    
    dfs_ssc_merged_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssc_merged_fillNA[n] = dfs_ssc_merged[n].replace('', np.nan)
        dfs_ssc_merged_fillNA[n] = dfs_ssc_merged_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssc_merged_fillNA[n].shape)
    
    #### loyaltyList
    
    dfs_l_fillNA = list(range(len(dimensions)))
    
    for n in range(len(dimensions)):
        dfs_l_fillNA[n] = dfs_l[n].replace('', np.nan)
        dfs_l_fillNA[n] = dfs_l_fillNA[n].replace('(not set)', np.nan)
        print(dfs_l_fillNA[n].shape)
    
    
    dfs_s_l_fillNA = list(range(len(dimensionsSpecial)))
    
    for n in range(len(dimensionsSpecial)):
        dfs_s_l_fillNA[n] = dfs_s_l[n].replace('', np.nan)
        dfs_s_l_fillNA[n] = dfs_s_l_fillNA[n].replace('(not set)', np.nan)
        print(dfs_s_l_fillNA[n].shape)
    
    
    dfs_ssd_l_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_l_fillNA[n] = dfs_ssd_l[n].replace('', np.nan)
        dfs_ssd_l_fillNA[n] = dfs_ssd_l_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssd_l_fillNA[n].shape)
    
    
    dfs_ssc_l_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssc_l_fillNA[n] = dfs_ssc_l[n].replace('', np.nan)
        dfs_ssc_l_fillNA[n] = dfs_ssc_l_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssc_l_fillNA[n].shape)
    
    #### activityList
    
    
    dfs_a_fillNA = list(range(len(dimensions)))
    
    for n in range(len(dimensions)):
        dfs_a_fillNA[n] = dfs_a[n].replace('', np.nan)
        dfs_a_fillNA[n] = dfs_a_fillNA[n].replace('(not set)', np.nan)
        print(dfs_a_fillNA[n].shape)
    
    
    dfs_s_a_fillNA = list(range(len(dimensionsSpecial)))
    
    for n in range(len(dimensionsSpecial)):
        dfs_s_a_fillNA[n] = dfs_s_a[n].replace('', np.nan)
        dfs_s_a_fillNA[n] = dfs_s_a_fillNA[n].replace('(not set)', np.nan)
        print(dfs_s_a_fillNA[n].shape)
    
    
    dfs_ssd_a_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssd_a_fillNA[n] = dfs_ssd_a[n].replace('', np.nan)
        dfs_ssd_a_fillNA[n] = dfs_ssd_a_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssd_a_fillNA[n].shape)
    
    
    dfs_ssc_a_fillNA = list(range(len(dimensionsSpecialSpecial)))
    
    for n in range(len(dimensionsSpecialSpecial)):
        dfs_ssc_a_fillNA[n] = dfs_ssc_a[n].replace('', np.nan)
        dfs_ssc_a_fillNA[n] = dfs_ssc_a_fillNA[n].replace('(not set)', np.nan)
        print(dfs_ssc_a_fillNA[n].shape)
    
    
    # ---------------------------------------------------------------------------------------------------------------------
        
    ## Dataframes to .csv
    
    csvFilePath = "output/csv/"
    fileName = dimensions[n]
    sep = ","
    
    for n in range(len(dfs_merged_fillNA)):
        csvFilePath = "output/csv/"
        fileName = dimensions[n]+"_dateHour"+"_cityId"+".csv" 
        appendDFToCSV(dfs_merged_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+".csv", sep)
    
    for n in range(len(dfs_s_merged_fillNA)):
        csvFilePath = "output/csv/"
        fileName = dimensionsSpecial[n]+"_date"+"_countryId"+".csv"
        appendDFToCSV(dfs_s_merged_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+".csv", sep)
    
    for n in range(len(dfs_ssd_merged_fillNA)):
        csvFilePath = "output/csv/"
        fileName = dimensionsSpecialSpecial[n]+"_date"".csv"
        appendDFToCSV(dfs_ssd_merged_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+".csv", sep)
    
    for n in range(len(dfs_ssc_merged_fillNA)):
        csvFilePath = "output/csv/"
        fileName = dimensionsSpecialSpecial[n]+"_countryId"+".csv"
        appendDFToCSV(dfs_ssc_merged_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+".csv", sep)
    
    for n in range(len(dfs_l_fillNA)):
        csvFilePath = "output/csv/loyalty/"
        fileName = dimensions[n]+"_dateHour"+"_cityId"+"_loyalty"+".csv"
        appendDFToCSV(dfs_l_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+"_loyalty"+".csv", sep)
    
    for n in range(len(dfs_s_l_fillNA)):
        csvFilePath = "output/csv/loyalty/"
        fileName = dimensionsSpecial[n]+"_date"+"_countryId"+"_loyalty"+".csv"
        appendDFToCSV(dfs_s_l_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+"_loyalty"+".csv", sep)
    
    for n in range(len(dfs_ssd_l_fillNA)):
        csvFilePath = "output/csv/loyalty/"
        fileName = dimensionsSpecialSpecial[n]+"_date"+"_loyalty"+".csv"
        appendDFToCSV(dfs_ssd_l_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+"_loyalty"+".csv", sep)
    
    for n in range(len(dfs_ssc_l_fillNA)):
        csvFilePath = "output/csv/loyalty/"
        fileName = dimensionsSpecialSpecial[n]+"_countryId"+"_loyalty"+".csv"
        appendDFToCSV(dfs_ssc_l_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+"_loyalty"+".csv", sep)
    
    for n in range(len(dfs_a_fillNA)):
        csvFilePath = "output/csv/activity/"
        fileName = dimensions[n]+"_dateHour"+"_cityId"+"_activity"+".csv"
        appendDFToCSV(dfs_a_fillNA[n], csvFilePath, dimensions[n]+"_dateHour"+"_cityId"+"_activity"+".csv", sep)
    
    for n in range(len(dfs_s_a_fillNA)):
        csvFilePath = "output/csv/activity/"
        fileName = dimensionsSpecial[n]+"_date"+"_countryId"+"_activity"+".csv"
        appendDFToCSV(dfs_s_a_fillNA[n], csvFilePath, dimensionsSpecial[n]+"_date"+"_countryId"+"_activity"+".csv", sep)
    
    for n in range(len(dfs_ssd_a_fillNA)):
        csvFilePath = "output/csv/activity/"
        fileName = dimensionsSpecialSpecial[n]+"_date"+"_activity"+".csv"
        appendDFToCSV(dfs_ssd_a_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_date"+"_activity"+".csv", sep)
    
    for n in range(len(dfs_ssc_a_fillNA)):
        csvFilePath = "output/csv/activity/"
        fileName = dimensionsSpecialSpecial[n]+"_countryId"+"_activity"+".csv"
        appendDFToCSV(dfs_ssc_a_fillNA[n], csvFilePath, dimensionsSpecialSpecial[n]+"_countryId"+"_activity"+".csv", sep)
    
    # ---------------------------------------------------------------------------------------------------------------------
        
    ## Date and Geography tables
    
    # Geography List
    geoList = ['continentId','continent','countryId','country','region','cityId','city']
    
    # Empty list to fill with dataframes
    
    report_request = gp.RunReportRequest(
        property=f"properties/{property_id}",
         dimensions=[
             gp.Dimension(name=geoList[0]),
             gp.Dimension(name=geoList[1]),
             gp.Dimension(name=geoList[2]),
             gp.Dimension(name=geoList[3]),
             gp.Dimension(name=geoList[4]),
             gp.Dimension(name=geoList[5]),
             gp.Dimension(name=geoList[6]),
           ],
           metrics=[
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
        
    # Perform query and append to list
    df_geo = gp.query(service_account, report_request, report_type="report")

    # Sort columns
    sortOrder = True
    df_geo = df_geo.sort_values(list(df_geo.columns.values), ascending=sortOrder)

    
    # Date List
    dateList = ["dateHour"]
    
    # Empty list to fill with dataframes
    
    report_request = gp.RunReportRequest(
        property=f"properties/{property_id}",
         dimensions=[
             gp.Dimension(name=dateList[0]),
           ],
           metrics=[
            ],
            date_ranges=[gp.DateRange(start_date=start_date, end_date=end_date)],
        )
    
    # Perform query and append to list
    df_dateHour = gp.query(service_account, report_request, report_type="report")

    # Sort out dates function
    df_dateHour = dates_funcs.sortOut_dateHour(df_dateHour)

    
    df_geoTest = df_geo.replace('', np.nan)
    df_geoTest = df_geoTest.replace('(not set)', np.nan)
    print(df_geoTest.shape)
    
    df_dateHourTest = df_dateHour.replace('', np.nan)
    df_dateHourTest = df_dateHourTest.replace('(not set)', np.nan)
    print(df_dateHourTest.shape)


    csvFilePath = "output/csv/reference/"
    fileName = "geographyInfo.csv"
    appendDFToCSV(df_geoTest, csvFilePath, fileName, sep)
    
    csvFilePath = "output/csv/reference/"
    fileName = "dateInfo.csv"
    appendDFToCSV(df_dateHourTest, csvFilePath, fileName, sep)

    # -------------------------------------------
    # Read in and merge geographyInfo with geographyCountryCodes 
    pathReferenceGeography="/Users/emil/miniforge3/envs/googleapi/Notebooks/Analytics/output/csv/reference/"
    #https://github.com/stefangabos/world_countries/blob/master/data/countries/en/world.csv
    fileNameGeoMaster = "countryCodes.csv"
    fileNameGeoInfo = "geographyInfo.csv"
    
    countryCodes = pathReferenceGeography+fileNameGeoMaster
    geoInfo = pathReferenceGeography+fileNameGeoInfo
    
    dfCountryCodes = pd.read_csv(countryCodes, dtype='object')
    dfGeoInfo = pd.read_csv(geoInfo, dtype='object')
    
    dfCountryCodes['alpha2'] = dfCountryCodes['alpha2'].str.upper()
    dfCountryCodes['alpha3'] = dfCountryCodes['alpha3'].str.upper()
    
    dfCountryCodes.rename(columns={'alpha2': 'countryId'}, inplace=True)
    dfCountryCodes.rename(columns={'alpha3': 'countryIdLong'}, inplace=True)
    dfCountryCodes.rename(columns={'name': 'countryRef'}, inplace=True)
    
    dfCountryCodes = dfCountryCodes.drop(columns=['id'])
    
    dfMerged = dfCountryCodes.merge(dfGeoInfo, how='outer', on='countryId')
    
    # Rearrange columns
    dfMerged = dfMerged[dfMerged.columns[[3,4,0,1,2,5,6,7,8]]]
    
    # Replace with NaN
    dfMerged = dfMerged.replace('', np.nan)
    dfMerged = dfMerged.replace('(not set)', np.nan)
    
    # Sort columns
    sortOrder = True
    dfSorted = dfMerged.sort_values(list(dfMerged.columns.values), ascending=sortOrder)
    
    sep = ','
    csvFilePath = "output/csv/reference/"
    fileName = "geographyReference.csv"
    fileNameFull = csvFilePath + fileName
    
    # Write to and overwrite file
    dfSorted.to_csv(fileNameFull, mode='w', index=0, sep=sep)        
    # -------------------------------------------

    
    ## Table with all dates, not just those with fetched data, as reference
    
    # Date range
    dates = pd.date_range('2022-01-01', '2030-01-01', freq="H",inclusive='left')
    
    # List to Dataframe
    dateReference = pd.DataFrame(dates, columns=['dateHour'])
    
    # Running function sortOut_dateHour
    dateReference = dates_funcs.sortOut_dateHour(dateReference)
    
    csvFilePath = "output/csv/reference/"
    fileName = "dateReference.csv"
    fileNameFull = csvFilePath + fileName
    
    # If file does not alreay exist, write to file
    if not os.path.isfile(fileNameFull):
        dateReference.to_csv(fileNameFull, mode='w', index=0, sep=sep)
    
    # ---------------------------------------------------------------------------------------------------------------------
    
    ## Max time to latestDatefile
    
    # Find largest date in dataframes, looking at df eventName, which should have all dates. (?)
    maxTimestamp = dfs_merged_fillNA[14]["dateFull"].max()
    maxTimestampString = datetime.datetime.strftime(maxTimestamp,'%Y-%m-%d')
    maxDate = datetime.datetime.strptime(maxTimestampString,'%Y-%m-%d').date()

    print("\n")
    print("Fetched data until "+maxTimestampString)
    
    # If date is larger than priviously max date.
    if maxDate > maxSavedDateDT:
    # Write largest date to file
        f = open("latestDate.txt", 'w')
        f.write(maxTimestampString)
        f.close()
    
    print("Written to .csv files")
    print("\n")

else:
    print("ERROR")

# ---------------------------------------------------------------------------------------------------------------------

# Footnote 1
print(
'''The following License applies to gapandas4 ONLY:

MIT License
Copyright (c) 2018
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the \"Software\"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED \"AS IS\", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

'''
)
# Now to print to log when script completed

nowDT = datetime.datetime.now()
now = datetime.datetime.strftime(nowDT,'%Y-%m-%d_%H:%M:%S')
print("Script finished: "+now)
print("\n")
print("--------------------------------------------------------------------------------")
print("\n")

Script started: 2023-06-16_01:17:10


Last date saved: 2023-06-14


Fetching new data, starting: 2023-06-15
up to and including: 2023-06-15
(83, 22)
(89, 22)
(85, 22)
(82, 22)
(87, 22)
(80, 22)
(92, 22)
(97, 22)
(74, 22)
(100, 22)
(103, 22)
(106, 22)
(97, 22)
(100, 22)
(253, 22)
(137, 22)
(78, 22)
(118, 22)
(125, 22)
(0, 20)
(0, 20)
(0, 19)
(0, 19)
(227, 6)
(279, 6)
(277, 6)
(243, 6)
(315, 6)
(386, 6)
(473, 6)
(556, 6)
(179, 6)
(623, 6)
(241, 6)
(270, 6)
(231, 6)
(261, 6)
(815, 6)
(709, 6)
(245, 6)
(595, 6)
(502, 6)
(0, 6)
(0, 6)
(0, 5)
(0, 5)
(489, 6)
(650, 6)
(624, 6)
(482, 6)
(764, 6)
(1028, 6)
(1379, 6)
(1525, 6)
(333, 6)
(1653, 6)
(467, 6)
(545, 6)
(457, 6)
(538, 6)
(1529, 6)
(1791, 6)
(510, 6)
(1495, 6)
(1246, 6)
(0, 6)
(0, 6)
(0, 5)
(0, 5)
output/csv/language_dateHour_cityId.csv
output/csv/languageCode_dateHour_cityId.csv
output/csv/browser_dateHour_cityId.csv
output/csv/deviceCategory_dateHour_cityId.csv
output/csv/mobileDeviceBranding_dateHour_cityId.csv
output/csv/mobileDevic